In [ ]:
# Test 
import os
import sys



pysur_path ="/home/gopalprabhu/Desktop/Surrogate--main/Surrogate_project/Surrogate_py/notebooks/"
sys.path.append(pysur_path)
import numpy as np
from joblib import Parallel, delayed
import lal
import pySurrogate as pysur
from tqdm import tqdm
import pickle
import warnings
from multiprocessing import freeze_support
warnings.filterwarnings("always")
from generate_Ft import Ftilde, PointLens
import time
import logging

# Configure logging
logging.basicConfig(
    filename='run_surr.log',  # Specify the log file name
    level=logging.DEBUG,       # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'


Ml=100.0
zl=0.5
y_min = 0.05
y_max = 1.0    #This was changed from 2 to 1
param_boundaries = [[y_min, y_max]]
times = np.arange(0.0, 30.0, 1 / 2048)           






def Ft(y, times, Ml, zl, dt_fac=5.0):
    point_lens = PointLens(y)
    ts = 8 * np.pi * lal.MTSUN_SI * Ml * (1 + zl)
    f_min, f_max = 10.0, 1000.0
    t_max = 30
    wmin = ts * f_min
    wmax = ts * f_max
    dt = min(np.pi / wmax / dt_fac, point_lens.img_td / dt_fac)  ##HAD NOTICED REDUCED ERRORS WITH INCREASED dt
    compute_Ft_params = {
        "lens": point_lens,
        "t_max": t_max,
        "dt": dt,
        "verbose": False,
    }
    t, Ft = Ftilde(**compute_Ft_params)
    FtNew = np.interp(times, t, Ft)
    #logging.debug("Ft calculated to be ", {Ft} )
    return FtNew
def Ft_shifted(y, times,t_ref=t_reference, Ml=100.0, zl=0.5, dt_fac=5.0):   #t_ref=4.58544921875
    print("Ft shifted function called with y =",y)
    point_lens = PointLens(y)
    #print(lal.MTSUN_SI,"<--M_sun", Ml,"<--ML",zl,"<--zl")
    ts = 8 * np.pi * lal.MTSUN_SI * 100.0 * (1 + 0.5)
    f_min, f_max = 10.0, 1000.0
    t_max = 30
    wmin = ts * f_min
    wmax = ts * f_max
    dt = min(np.pi / wmax / dt_fac, point_lens.img_td / dt_fac)  ##HAD NOTICED REDUCED ERRORS WITH INCREASED dt
    compute_Ft_params = {
        "lens": point_lens,
        "t_max": t_max,
        "dt": dt,
        "verbose": False,
    }
    t, Ft = Ftilde(**compute_Ft_params)
    
    FtNew=np.interp(times, t, Ft)
    peak_index=np.argmax(FtNew)
    t_peak=times[peak_index]                                                      #Find the peak time
    t_shift=np.abs(t_ref-t_peak)                                                    #Fine how much to shift the time to align the peak with t_ref
    print("t_shift is:",t_shift)
        
    
    
    Ft_extended=[]
    for time in new_extended_times:
        if time<t_shift:                                              # find t_shift of the rightmost shifted Ft function 
            Ft_extended.extend([0])
        elif time>30.0+t_shift:
            Ft_extended.extend([0])

        else:
            Ft_extended.extend([FtNew[np.argmin(np.abs(times-np.abs((time -t_shift))))]])    
            #print("This is the result of the Ft_extended function:", Ft_extended)
    print("Ft extended was calculated and returned")        
    return Ft_extended